# Predictions
### Steps : 
* Train multiple models on the dataset  
* Evaluate them  
* Select the bests models
* Hyperparameter tuning
* Ensemble training

## Importing dataset and libraries

In [123]:
import pandas as pd
import numpy as np
### Preprocessing libraries
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
### Models
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
df = pd.read_csv('/content/clean_data.csv')

### Hyperparameter tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

## Prepare data for the training




### Split X and y

In [65]:
X = df.drop(["Div","Date","FTR", "FTHG", "FTAG", "HTHG", "HTAG", "HTR", "HS", "AS", "AST", "HF", "AF", "HC",'AC', 'HY', 'AY', 'HR', 'AR', "Numerical_ftr", "Numerical_htr"], axis=1)
y = df["FTR"]

Establish categorical columns and numerical columns

In [66]:
categorical_features = [col for col in X_train.select_dtypes(include='object')]
numerical_features = [col for col in X_train.select_dtypes(include='float64')]

Instantiate preprocessing pipelines

In [67]:
num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
cat_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore"))
full_pipeline = make_column_transformer((num_pipeline, numerical_features), (cat_pipeline, categorical_features))

Split train/test data

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

## Training the models

In [103]:
log_reg = LogisticRegression(max_iter=500)
logistic_regression = make_pipeline(full_pipeline, log_reg)

xgb = XGBClassifier(n_estimators=100)
xgb_model = make_pipeline(full_pipeline, xgb)

decision_tree = DecisionTreeClassifier(max_depth=20, min_samples_split=13)
decision_model = make_pipeline(full_pipeline, decision_tree)

models = [(xgb_model, "xgb"), (logistic_regression, "logistic_regression"), (decision_model, "decision tree")]

In [104]:
for model in models : 
  model[0].fit(X_train, y_train)
  score = model[0].score(X_test, y_test)
  print(score)
  print(model[1])

0.5482456140350878
xgb
0.5394736842105263
logistic_regression
0.4451754385964912
decision tree


The best models are XGB and logistic regression

## Hyperparameter tuning with grid-search

#### Logistic regression gridsearch

In [ ]:
param_grid = [
              {"logisticregression__penalty" : ["l1", "l2", "elasticnet"], 
               "logisticregression__solver": ["newton-cg", "lbfgs", "liblinear", "sag", "sag "], 
               "logisticregression__C":[100, 10, 1.0, 0.1, 0.01], "logisticregression__max_iter":[500]}
]
log_model = LogisticRegression()
logistic_model = make_pipeline(full_pipeline, log_model)
grid_search = GridSearchCV(logistic_model, param_grid, cv=10)
grid_search.fit(X_train, y_train)


In [118]:
grid_search.best_score_

0.5822134150003002

In [119]:
grid_search.best_params_

{'logisticregression__C': 0.01,
 'logisticregression__max_iter': 500,
 'logisticregression__penalty': 'l1',
 'logisticregression__solver': 'liblinear'}

In [120]:
best_linear_regression = LogisticRegression(C=0.01, max_iter=500, penalty='l1', solver="liblinear")

#### Xgboost gridsearch


In [128]:
params = { 'xgbclassifier__max_depth': [3, 5, 6, 10, 15, 20, 25, 30],
           'xgbclassifier__learning_rate': [0.01, 0.1, 0.2, 0.3],
           'xgbclassifier__subsample': np.arange(0.5, 1.0, 0.1),
           'xgbclassifier__colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'xgbclassifier__colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'xgbclassifier__n_estimators': [100,300,400,500,600, 700, 800, 1000]}
xgb = XGBClassifier()
xgb_grid_model = make_pipeline(full_pipeline, xgb)
random_grid = RandomizedSearchCV(xgb_grid_model, params, cv=10)
random_grid.fit(X_train, y_train)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('columntransformer',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('pipeline-1',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('simpleimputer',
                                                                                                SimpleImputer(add_indicator=False,
                                                                  

In [130]:
random_grid.best_params_

{'xgbclassifier__colsample_bylevel': 0.6,
 'xgbclassifier__colsample_bytree': 0.8999999999999999,
 'xgbclassifier__learning_rate': 0.01,
 'xgbclassifier__max_depth': 3,
 'xgbclassifier__n_estimators': 300,
 'xgbclassifier__subsample': 0.7999999999999999}

In [132]:
best_xgb_model = XGBClassifier(colsample_bylevel=0.6, 
                               colsample_bytree=0.8999999999999999, 
                               learning_rate=0.01, max_depth=3, 
                               n_estimators=300, 
                               subsample=0.7999999999999999)

## Ensemble training for the best models
* VotingClassifier with XGBClassifier and LogisticRegression
* BaggingClassifier With XGBClassifier / BaggingClassifier with LogisticRegression